# Dog breed classification using PyTorch and AWS SageMAker

This notebook contains steps and code that uses AWS SageMaker to finetune a pretrained RESNET50 model to perform image classification on dogs, classifying them according to their breeds. We will make use of SageMaker's Debugger to debug, monitor and profile the training job.  ----ADD MORE DETAILS- maybe about the network used, the library used and other details----

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.

**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

In [31]:
!pip install smdebug


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [32]:
import sagemaker
import boto3

## Dataset

The dataset used in this project is a labeled dataset of 8,351 real-world images of 133 American Kennel Club (AKC) recognized dog breeds. It is divided into 6680 images for training, 836 images for testing, 835 images for validation.  
The images in this dataset are of different sizes and backgrounds. The number of images provided for each breed varies, some breeds have fewer images than others.

In [ ]:
# Command to download and unzip data
!wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
!unzip dogImages.zip

In [5]:
# Upload the data to Amazon S3

import os
from sagemaker import get_execution_role

bucket= 'sagemaker-us-east-1-519574148523'
region = 'us-east-1'
role =  get_execution_role()
print("Default Bucket: {}".format(bucket))
print("AWS Region: {}".format(region))
print("RoleArn: {}".format(role))

os.environ["DEFAULT_S3_BUCKET"] = bucket
!aws s3 sync ./dogImages s3://${DEFAULT_S3_BUCKET}/data/

Default Bucket: sagemaker-us-east-1-519574148523
AWS Region: us-east-1
RoleArn: arn:aws:iam::519574148523:role/service-role/AmazonSageMaker-ExecutionRole-20230209T103282


## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [33]:
# Declare your HP ranges, metrics etc.
from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

hyperparameter_ranges = {
    "lr": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128])
}
#objective_metric_name='validation:accuracy'
#metric_definitions = [{'Name': objective_metric_name, 'Regex': 'accuracy=(\d+\.\d+)'}]
objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [34]:
#Create estimators for your HPs
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
		    entry_point='hpo.py',
            role=role,
            framework_version='1.8.1',
            py_version='py3',
            instance_count=1,
            instance_type='ml.m5.large',
		)

tuner = HyperparameterTuner( estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=1,
    max_parallel_jobs=1,
    objective_type=objective_type,
)

In [29]:
# Set certain OS variables
os.environ['SM_MODEL_DIR']='s3://{}/model/'.format(bucket)
os.environ['SM_CHANNEL_TRAIN']='s3://{}/data/'.format(bucket)
#os.environ['SM_OUTPUT_DATA_DIR']='s3://{}/output/'.format(bucket)

In [30]:
# Fit your HP Tuner
tuner.fit({'train': "s3://{}/data/".format(bucket)},wait=True) 

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


............................................................................................................................................................

KeyboardInterrupt: 

In [15]:
# Get the best estimators and the best HPs

best_estimator = tuner.best_training_job()

#Get the hyperparameters of the best trained model
best_estimator.hyperparameters()

Exception: Best training job not available for tuning job: pytorch-training-230223-1249

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()